In [ ]:
import numpy as np

In [ ]:
# Environment and Policy Setup
# State space: S0 (Start), S_A (Target), S_B, S_C, S_D (Terminal)
# Rewards are obtained upon transitioning to the next state.
STATES = ['S0', 'S_A', 'S_B', 'S_C']
TERMINAL_STATE = 'S_D'